# Document Chracterizzation


## Import of relevant libraries 

In [58]:
# Import of all the libraries used for the project
import os
import shutil
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
from operator import itemgetter

## Dataset import

In [6]:
## import the CSV file with the labels

data_input = pd.read_csv('Document Characterisation Data Dump.csv', sep=',')
print(len(data_input["URL"]))
data_input


129


,URL,CLASS
0,https://issafrica.org/iss-today/time-to-tackle...,Analytical Report
1,http://www.understandingwar.org/backgrounder/r...,Analytical Report
2,http://www.understandingwar.org/backgrounder/r...,Analytical Report
3,http://www.understandingwar.org/backgrounder/w...,Analytical Report
4,http://www.understandingwar.org/backgrounder/r...,Analytical Report
...,...,...
124,https://rewardsforjustice.net/english/muhammad...,Profile Report
125,https://rewardsforjustice.net/english/abdelkar...,Profile Report
126,https://rewardsforjustice.net/english/muhammad...,Profile Report
127,https://rewardsforjustice.net/english/jamal_ra...,Profile Report


In [8]:
# Defined a simple scraper  to go through all the link, visit the page and copy all the txt in the paragraph tag of the HTML file

def scraper(input):
    # create an empty array
    content = []

    # for loop on all the links in the dataframe
    for i in range(len(input["URL"])):
        url = input["URL"][i]
        # text = url
        text = ' '

        # check if actually the url is correct orr not
        if not (url.startswith('https://') or url.startswith('http://')):
            text += ' Link not good'
            content.append(text) 
            continue
        
        # Request the url and copy of the HTMl 
        page = requests.get(url)
        
        # Check if the response is positive or not, if not set that page was not found 
        if page.status_code != 200:
            text += ' Page not able to find'

        # Else take the text from all the p tag of the HTML content  
        else:
            soup = BeautifulSoup(page.content, 'html.parser')
            for p in soup.find_all('p'):
                text = text + ' ' + p.text
        
        content.append(text) 
    return content

In [9]:
# Calling the scraper function
data = scraper(data_input)

# Adding the scraped data into the Dataframe  
data_input['Content'] = data
data_input

,URL,CLASS,Content
0,https://issafrica.org/iss-today/time-to-tackle...,Analytical Report,"2020-11-25 On 26 September, hundreds of Mali..."
1,http://www.understandingwar.org/backgrounder/r...,Analytical Report,"By George Barros November 16, 2020 The Kreml..."
2,http://www.understandingwar.org/backgrounder/r...,Analytical Report,"By Ezgi Yazici November 13, 2020 Key Takeawa..."
3,http://www.understandingwar.org/backgrounder/w...,Analytical Report,By Katherine Lawlor with Brandon Wallace Key...
4,http://www.understandingwar.org/backgrounder/r...,Analytical Report,By Isabel Ivanescu and Ezgi Yazici Key Takea...
...,...,...,...
124,https://rewardsforjustice.net/english/muhammad...,Profile Report,"If you prefer a different language, you may ..."
125,https://rewardsforjustice.net/english/abdelkar...,Profile Report,"If you prefer a different language, you may ..."
126,https://rewardsforjustice.net/english/muhammad...,Profile Report,"If you prefer a different language, you may ..."
127,https://rewardsforjustice.net/english/jamal_ra...,Profile Report,"If you prefer a different language, you may ..."


## Experimental Data Analysis 

In [44]:
corpus = data_input['Content'][0]

# tockenizzation of the corpus 
def tokenize(corpus):

    # The backslash is an escape character to let the interpreter know we mean to use it as a string literal (- or ') 
    re_punctuation_string = '[\s,/.?\-\']'  # split on spaces (\s),  commas (,), slash (/), fullstop (.), question marks (?), hyphens (-), and apostrophe (').
    tokenized = re.split(re_punctuation_string, corpus) # in python's regex, [...] is an alternative to writing .|.|.
    tokenized = list(filter(None, tokenized)) # remove empty strings from list 
    tokenized = [token.lower() for token in tokenized] # Lowercasing everything

    return tokenized    

        
tokenized_corpus = tokenize(corpus)
print(tokenized_corpus)

['2020', '11', '25', 'on', '26', 'september', 'hundreds', 'of', 'malian', 'women', 'and', 'some', 'men', 'rallied', 'in', 'bamako', 'to', 'protest', 'against', 'the', 'trivialisation', 'of', 'violence', 'against', 'women', 'this', 'was', 'prompted', 'by', 'the', 'revelation', 'on', 'social', 'media', 'of', 'alleged', 'intimate', 'partner', 'abuse', 'involving', 'a', 'famous', 'malian', 'artist', 'the', 'story', 'broke', 'just', 'two', 'months', 'before', 'the', '25', 'november', 'annual', 'international', 'day', 'for', 'the', 'elimination', 'of', 'violence', 'against', 'women', 'the', 'case', 'now', 'pending', 'in', 'court', 'sheds', 'light', 'on', 'a', 'society', 'wide', 'problem', 'that’s', 'often', 'silenced', 'by', 'deeply', 'entrenched', 'social', 'norms', 'and', 'religious', 'discourse', 'fear', 'on', 'the', 'part', 'of', 'the', 'political', 'elite', 'to', 'confront', 'the', 'divisive', 'issue', 'makes', 'matters', 'worse', 'official', 'figures', 'on', 'violence', 'against', 'wom

In [57]:
# function to vount the tockenization
def distinct_tokens_count(tokenized_corpus):

    distinct_tokens_count = {}
    for t_review in tokenized_corpus:

        if t_review not in distinct_tokens_count.keys():
            distinct_tokens_count[t_review] = 1
        else:
            distinct_tokens_count[t_review] += 1
            
    # Sort the key value to greter to smaller
    distinct_tokens_count = sorted(distinct_tokens_count.items(), key=itemgetter(1), reverse=True) 

    for token, count in distinct_tokens_count:
        print("{:<14s} {:<10d}".format(token, count))
    return distinct_tokens_count

a = distinct_tokens_count(tokenized_corpus)
print(a)

[('the', 127), ('and', 83), ('to', 70), ('of', 69), ('women', 62), ('in', 62), ('a', 44), ('violence', 36), ('against', 30), ('on', 28), ('for', 28), ('this', 26), ('by', 26), ('be', 22), ('women’s', 20), ('law', 18), ('rights', 16), ('that', 14), ('will', 14), ('transition', 12), ('was', 10), ('but', 10), ('gender', 10), ('national', 10), ('transitional', 10), ('all', 10), ('an', 10), ('agenda', 10), ('need', 10), ('also', 9), ('social', 8), ('religious', 8), ('political', 8), ('are', 8), ('mali', 8), ('government', 8), ('groups', 8), ('areas', 8), ('justice', 8), ('leadership', 8), ('it', 8), ('public', 8), ('malian', 6), ('men', 6), ('partner', 6), ('two', 6), ('society', 6), ('more', 6), ('country’s', 6), ('based', 6), ('little', 6), ('including', 6), ('conflict', 6), ('sexual', 6), ('as', 6), ('with', 6), ('been', 6), ('improving', 6), ('africa', 6), ('had', 6), ('physical', 6), ('organisations', 6), ('have', 6), ('wasn’t', 6), ('pressure', 6), ('powerful', 6), ('from', 6), ('impr

{'one': 5, 'three': 2, 'five': 1, 'two': 10, 'four': 8}
[('two', 10), ('four', 8), ('one', 5), ('three', 2), ('five', 1)]


Pre trained BERT models

https://pypi.org/project/pytorch-pretrained-bert/

https://pytorch.org/hub/huggingface_pytorch-transformers/

https://medium.com/@aniruddha.choudhury94/part-2-bert-fine-tuning-tutorial-with-pytorch-for-text-classification-on-the-corpus-of-linguistic-18057ce330e1
